In [16]:
from __future__ import print_function

import glob
import math
import os

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.data import Dataset
%matplotlib inline

# tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

mnist_dataframe = pd.read_csv("data/train.csv", sep=",")

# Use just the first 10,000 records for training/validation.
head = 5000 # Data size
mnist_dataframe = mnist_dataframe.head(head)#10000)
valid = head//10 # Validation size

mnist_dataframe = mnist_dataframe.reindex(np.random.permutation(mnist_dataframe.index))
mnist_dataframe.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1620,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2112,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3245,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
958,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1907,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
mnist_dataframe.iloc[:, 72:73]#.describe()

,pixel71
1620,0
2112,0
3245,0
958,0
1907,255
...,...
3805,0
3104,0
4903,0
114,0


In [18]:
def parse_labels_and_features(dataset):
  """Extracts labels and features.
  
  This is a good place to scale or transform the features if needed.
  
  Args:
    dataset: A Pandas `Dataframe`, containing the label on the first column and
      monochrome pixel values on the remaining columns, in row major order.
  Returns:
    A `tuple` `(labels, features)`:
      labels: A Pandas `Series`.
      features: A Pandas `DataFrame`.
  """
  labels = dataset['label']

  # DataFrame.loc index ranges are inclusive at both ends.
  features = dataset.iloc[:,1:785]
  # Scale the data to [0, 1] by dividing out the max value, 255.
#   features = features // 255
  features = features.applymap(lambda x: x / 255)

  return labels, features

In [19]:
def parse_test_labels_and_features(labels1, dataset):
  """Extracts labels and features.
  
  This is a good place to scale or transform the features if needed.
  
  Args:
    dataset: A Pandas `Dataframe`, containing the label on the first column and
      monochrome pixel values on the remaining columns, in row major order.
  Returns:
    A `tuple` `(labels, features)`:
      labels: A Pandas `Series`.
      features: A Pandas `DataFrame`.
  """
  labels = labels1['Label']

  # DataFrame.loc index ranges are inclusive at both ends.
  features = dataset.copy()
  # Scale the data to [0, 1] by dividing out the max value, 255.
#   features = features // 255
#   features = features.applymap(lambda x: x / 255)

  return labels, features

In [20]:
def adjust_image(data):
    # Reshape to [batch, height, width, channels].
    imgs = tf.reshape(data, [-1, 28, 28, 1])
    # Adjust image size to Inception-v3 input.
    imgs = tf.image.resize_images(imgs, (96,96))#(299, 299))
    # Convert to RGB image.
    imgs = tf.image.grayscale_to_rgb(imgs)
    # Convert to numpy array
    imgs = tf.Session().run(imgs)
    return imgs

In [21]:
targets, examples = parse_labels_and_features(mnist_dataframe)
examples = adjust_image(examples.values)
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
#     examples = np.array(examples.eval()) #changes from tensor to np.array
# examples = np.array(tf.Session().run(examples).eval)
print(examples.shape)

(5000, 96, 96, 3)


In [22]:
print(examples)

training_targets, training_examples = targets[:-valid], examples[:-valid]
# training_targets, training_examples = parse_labels_and_features(mnist_dataframe[:750*10])#divided by 10
training_examples#.describe()

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 

In [23]:
validation_targets, validation_examples = targets[-valid:], examples[-valid:]
# validation_targets, validation_examples = parse_labels_and_features(mnist_dataframe[750*10:1000*10])#divided by 10
# print(validation_examples.shape)#.describe()
print(examples.max())

1.0


In [24]:
# rand_example = np.random.choice(training_examples.index)
# _, ax = plt.subplots()
# # ax.imshow(training_examples.loc[rand_example].values.reshape(28, 28))
# ax.matshow(training_examples.loc[rand_example].values.reshape(28, 28))#pic size (128,128)
# ax.set_title("Label: %i" % training_targets.loc[rand_example])
# ax.grid(False)

In [25]:
# def construct_feature_columns():
#   """Construct the TensorFlow Feature Columns.

#   Returns:
#     A set of feature columns
#   """ 
  
#   # There are 784 pixels in each image.
#   return set([tf.feature_column.numeric_column('x', shape=784)])

In [26]:
def create_training_input_fn(features, labels, batch_size):#, num_epochs=None, shuffle=True):

  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x=features,
      y=labels,
      batch_size=batch_size,
      num_epochs=None,
      shuffle=True)

  return train_input_fn

In [27]:
def create_predict_input_fn(features, labels, batch_size):

  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x=features,
      y=labels,
      num_epochs=1,
      shuffle=False)

  return predict_input_fn

In [28]:
def mobilenet_model_fn(features, labels, mode, params):
    # Load mobilenet model.
    module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_035_96/feature_vector/1")#imagenet/inception_v3/feature_vector/1")
#     module = params["module"]
#     input_layer = adjust_image(features["x"])
#     outputs = module(input_layer)
    input_layer = features
    outputs = module(input_layer)

    logits = tf.layers.dense(inputs=outputs, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])#0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [29]:
def train_tfhub_classification_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
#     module,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a neural network classification model for the MNIST digits dataset.
  
  In addition to training, this function also prints training progress information,
  a plot of the training and validation loss over time, as well as a confusion
  matrix.
  
  Args:
    learning_rate: An `int`, the learning rate to use.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing the training features.
    training_targets: A `DataFrame` containing the training labels.
    validation_examples: A `DataFrame` containing the validation features.
    validation_targets: A `DataFrame` containing the validation labels.
      
  Returns:
    The trained `Classifier` object.
  """

#   ## Data logging in terminal
#   tf.logging.set_verbosity(tf.logging.INFO)
#         # Set up logging for predictions
#   tensors_to_log = {"probabilities": "softmax_tensor"}
#   logging_hook = tf.train.LoggingTensorHook(
#             tensors=tensors_to_log, every_n_iter=10)    

  periods = 10
  # Caution: input pipelines are reset with each call to train. 
  # If the number of steps is small, your model may never see most of the data.  
  # So with multiple `.train` calls like this you may want to control the length 
  # of training with num_epochs passed to the input_fn. Or, you can do a really-big shuffle, 
  # or since it's in-memory data, shuffle all the data in the `input_fn`.
  steps_per_period = steps / periods  
  
  # Create the input functions.
  predict_training_input_fn = create_predict_input_fn(
    training_examples, training_targets, batch_size)
  predict_validation_input_fn = create_predict_input_fn(
    validation_examples, validation_targets, batch_size)
  training_input_fn = create_training_input_fn(
    training_examples, training_targets, batch_size)

  # Create feature columns.
#   feature_columns = [tf.feature_column.numeric_column('pixels', shape=784)]

#   my_optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
#   my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

#   my_optimizer = tf.train.AdagradOptimizer(learning_rate=0.05)
#   my_optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
#   my_optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    
    
  # Create the Estimator                        ## ADDED params so that I can pass values to model
  classifier = tf.estimator.Estimator(
    model_fn=mobilenet_model_fn, model_dir="/tmp/mobilenet_v6", params={
        'learning_rate': learning_rate,
        'optimizer' : my_optimizer,
#         'module' : module
#         'feature_columns': my_feature_columns,
#         # Two hidden layers of 10 nodes each.
#         'hidden_units': [10, 10],
#         # The model must choose between 3 classes.
#         'n_classes': 3,
    })

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("LogLoss error and accuracy (on validation data):")
  training_errors = []
  validation_errors = []
  for period in range (0, periods):
  # Train the model, starting from the prior state.

  # Train the model
    classifier.train(
          input_fn=training_input_fn,
          steps=steps_per_period,#,#000,#20000,
#           hooks=[logging_hook]
        )
    
    eval_results = classifier.evaluate(input_fn=predict_validation_input_fn)
    print(eval_results)
  
#     # Take a break and compute probabilities.
#     training_predictions = list(classifier.predict(input_fn=predict_training_input_fn))
#     training_pred_class_id = np.array([item['classes'] for item in training_predictions])
#     training_pred_one_hot = tf.keras.utils.to_categorical(training_pred_class_id,10)
       
#     validation_predictions = list(classifier.predict(input_fn=predict_validation_input_fn)) 
#     validation_pred_class_id = np.array([item['classes'] for item in validation_predictions])
#     validation_pred_one_hot = tf.keras.utils.to_categorical(validation_pred_class_id,10)   
    
#     # Compute training and validation errors.
#     training_log_loss = metrics.log_loss(training_targets, training_pred_one_hot)
#     validation_log_loss = metrics.log_loss(validation_targets, validation_pred_one_hot)
#     # Occasionally print the current loss.
#     print("  period %02d : %0.2f     Accuracy: %02f" % (period, validation_log_loss, eval_results['accuracy']))
#     # Add the loss metrics from this period to our list.
#     training_errors.append(training_log_loss)
#     validation_errors.append(validation_log_loss)
#   print("Model training finished.")
#   # Remove event files to save disk space.
#   _ = map(os.remove, glob.glob(os.path.join(classifier.model_dir, 'events.out.tfevents*')))
  
#   # Calculate final predictions (not probabilities, as above).
#   final_predictions = classifier.predict(input_fn=predict_validation_input_fn)
#   final_predictions = np.array([item['classes'] for item in final_predictions])
  
  
#   accuracy = metrics.accuracy_score(validation_targets, final_predictions)
#   print("Final accuracy (on validation data): %0.2f" % accuracy)

#   # Output a graph of loss metrics over periods.
#   plt.ylabel("LogLoss")
#   plt.xlabel("Periods")
#   plt.title("LogLoss vs. Periods")
#   plt.plot(training_errors, label="training")
#   plt.plot(validation_errors, label="validation")
#   plt.legend()
#   plt.show()
  
#   # Output a plot of the confusion matrix.
#   cm = metrics.confusion_matrix(validation_targets, final_predictions)
#   # Normalize the confusion matrix by row (i.e by the number of samples
#   # in each class).
#   cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
#   ax = sns.heatmap(cm_normalized, cmap="bone_r")
#   ax.set_aspect(1)
#   plt.title("Confusion matrix")
#   plt.ylabel("True label")
#   plt.xlabel("Predicted label")
#   plt.show()

  return classifier

In [30]:
# print(training_examples)
# raw_training_examples = {"x": training_examples.values}
# raw_validation_examples = {"x": validation_examples.values}
# new_training_examples = adjust_image(np.array(raw_training_examples))
# new_validation_examples = adjust_image(np.array(raw_validation_examples))
# module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/2")


tfhub_classifier = train_tfhub_classification_model(
    learning_rate=.005,#0.001,#0.05,
    steps=100,#000,#1000,
    batch_size=10,
#     module=module,
    hidden_units=[5],#[100, 100],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Training model...
LogLoss error and accuracy (on validation data):


InvalidArgumentError: Received a label value of -9223372036854775808 which is outside the valid range of [0, 10).  Label values: 4 1 8 4 8 9 2 5 9 -9223372036854775808
	 [[Node: sparse_softmax_cross_entropy_loss/xentropy/xentropy = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense/BiasAdd, random_shuffle_queue_DequeueMany:2)]]

Caused by op 'sparse_softmax_cross_entropy_loss/xentropy/xentropy', defined at:
  File "/home/harrisonfsmith95/anaconda3/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/harrisonfsmith95/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/harrisonfsmith95/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/harrisonfsmith95/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_insta

In [ ]:
mnist_test_dataframe = pd.read_csv(
  "data/test.csv",
  sep=",")

mnist_test_sample_dataframe = pd.read_csv(
  "data/sample_submission.csv",
  sep=",")

# test_targets, test_examples = parse_test_labels_and_features(mnist_test_sample_dataframe, mnist_test_dataframe)#parse_labels_and_features(mnist_test_dataframe)


# test_examples =   features = dataset.iloc[:,1:785]
  # Scale the data to [0, 1] by dividing out the max value, 255.
#   features = features // 255
#   features = features.applymap(lambda x: x / 255)

test_examples = mnist_test_dataframe.copy().applymap(lambda x: x / 255)
# print(mnist_test_sample_dataframe)
test_targets = mnist_test_sample_dataframe['Label'].copy()
test_examples.describe()
# test_targets.describe()

In [ ]:
predict_test_input_fn = create_predict_input_fn(
    test_examples, test_targets, batch_size=100)

test_predictions = tfhub_classifier.predict(input_fn=predict_test_input_fn)
test_predictions = np.array([item['classes'] for item in test_predictions])
  
accuracy = metrics.accuracy_score(test_targets, test_predictions)
print("Accuracy on test data: %0.2f" % accuracy)

In [ ]:
print(len(test_predictions))
new = pd.DataFrame(test_predictions, columns=["Label"])#[1], index=new_predictions[0])
# print(new)
# new.index.name = "ImageId"
new = pd.concat([mnist_test_sample_dataframe['ImageId'], new], axis=1)
print(new)

In [ ]:
new.to_csv('predictions_half_data_vis50000steps1.csv', index=False)

In [ ]:
# def train_tfhub_classification_model_quick(
#     learning_rate,
#     steps,
#     batch_size,
#     hidden_units,
#     training_examples,
#     training_targets,
#     validation_examples,
#     validation_targets):
#   """Trains a neural network classification model for the MNIST digits dataset.
  
#   In addition to training, this function also prints training progress information,
#   a plot of the training and validation loss over time, as well as a confusion
#   matrix.
  
#   Args:
#     learning_rate: An `int`, the learning rate to use.
#     steps: A non-zero `int`, the total number of training steps. A training step
#       consists of a forward and backward pass using a single batch.
#     batch_size: A non-zero `int`, the batch size.
#     hidden_units: A `list` of int values, specifying the number of neurons in each layer.
#     training_examples: A `DataFrame` containing the training features.
#     training_targets: A `DataFrame` containing the training labels.
#     validation_examples: A `DataFrame` containing the validation features.
#     validation_targets: A `DataFrame` containing the validation labels.
      
#   Returns:
#     The trained `CNNClassifier` object.
#   """
#   ## Data logging in terminal###########################added
#   tf.logging.set_verbosity(tf.logging.INFO)

#   # Set up logging for predictions
#   tensors_to_log = {"loss" : loss, "accuracy" : accuracy}
# #   tensors_to_log = {"probabilities": "softmax_tensor"}
#   logging_hook = tf.train.LoggingTensorHook(
#             tensors=tensors_to_log, every_n_iter=10)    
  
#   # Create the input functions.
#   predict_training_input_fn = create_predict_input_fn(
#     training_examples, training_targets, batch_size)
#   predict_validation_input_fn = create_predict_input_fn(
#     validation_examples, validation_targets, batch_size)
#   training_input_fn = create_training_input_fn(
#     training_examples, training_targets, batch_size)
    
    
#   # Create the Estimator                        ## ADDED params so that I can pass values to model
#   classifier = tf.estimator.Estimator(
#     model_fn=inceptionv3_model_fn, model_dir="/tmp/mobilenet_v2/6", params={
#         'learning_rate': learning_rate,
# #         'optimizer' : my_optimizer,
# #         'feature_columns': my_feature_columns,
# #         # Two hidden layers of 10 nodes each.
# #         'hidden_units': [10, 10],
# #         # The model must choose between 3 classes.
# #         'n_classes': 3,
#     })

#   # Train the model, but do so inside a loop so that we can periodically assess
#   # loss metrics.
#   print("Training model...")

#     # Train the model
#   classifier.train(
#           input_fn=training_input_fn,
#           steps=steps,#,#000,#20000,
#           hooks=[logging_hook]#training_hooks
#         )

#   eval_results = classifier.evaluate(input_fn=predict_validation_input_fn)
#   print(eval_results)
#   print("Finished!")
#   return classifier

In [ ]:
# tfhub_classifier = train_tfhub_classification_model_quick(
#     learning_rate=1,#.5,#0.001,#0.05,
#     steps=5000,#5000,#000,#1000,
#     batch_size=100,
#     hidden_units=[5],#[100, 100],
#     training_examples=training_examples,
#     training_targets=training_targets,
#     validation_examples=validation_examples,
#     validation_targets=validation_targets)